# 환경설정

In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm
import pandas as pd
import sys

In [51]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import html5lib
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException

# 함수 정의

In [52]:
##################################################################################################################################################
# Functions 
##################################################################################################################################################
# 스크롤 다운
def scr_down(limit):
    scr_cnt = 0
    while True:    
        # 한 번 스크롤 다운 
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        
        time.sleep(0.2)

        # 스크롤 다운 카운트 
        scr_cnt += 1
        if scr_cnt % 20 == 0:
            print('>', end='')
        
        if scr_cnt == limit:
            break
        
    print(' Scroll Down End!')
    
# 로그인
def login(ID, PW):
    # 로그인 클릭
    driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-1xm32e0 > header.css-6k8tqb > nav > div > div > ul > li:nth-child(7) > button').click()
    
    time.sleep(1)

    # 로그인 아이디, 비밀번호 입력
    login_box = driver.find_elements(By.CLASS_NAME, 'css-1oyrbwm')
    login_box[0].send_keys(ID)
    login_box[1].send_keys(PW)
    
    # 로그인 클릭
    login_box[1].send_keys(Keys.ENTER)   # 엔터키 입력

# 로그아웃
def logout():
    # 마이페이지
    driver.find_element(By.CLASS_NAME, 'css-ipiine').click()
    time.sleep(0.5)
    
    # 환경설정
    driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-1xm32e0 > section > div > div > div > div > section > div.css-ewkqyn-WallPaper.e19zkogf3 > button').click()
    
    # 로그아웃 클릭
    driver.find_elements(By.CLASS_NAME, 'css-1ciqeix')[-2].click()
    
    # 확인 클릭
    driver.find_elements(By.CLASS_NAME, 'css-gy4ioq')[-1].click()

# 데이터 불러오기

In [53]:
data = pd.read_csv('data/KOBIS_TOP100.csv', encoding='utf-8')

m_list = data['영화명']
m_year = data['링크연도']
m_year[6]='2014'
m_year[14]='2018'
m_year[34]='2013'
m_year[37]='2022'
m_year[41]='2017'
m_year[72]='2015'
m_year[74]='2014'
m_year[98]='2017'

m_list[40]='디 워'
m_list[45]='미션 임파서블 : 고스트 프로토콜'
m_list[51]='한산 리덕스'
m_list[52]='스파이더맨: 홈커밍'
m_list[64]='곡성(哭聲)'

m_year = m_year.astype('int')


C:\Users\narae\AppData\Local\Temp\ipykernel_20756\67878278.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_year[6]='2014'
C:\Users\narae\AppData\Local\Temp\ipykernel_20756\67878278.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_list[40]='디 워'
C:\Users\narae\AppData\Local\Temp\ipykernel_20756\67878278.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_list[45]='미션 임파서블 : 고스트 프로토콜'
C:\Users\narae\AppData\Local\Temp\ipyk

# 크롬 열기

In [54]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 영화 사이트 접속 & 로그인

In [55]:
# 영화 리뷰 가져오기
# 사이트 접속
driver.get('https://pedia.watcha.com/ko-KR')

time.sleep(1)

# 로그인 하기
ID = '아이디를 입력하세요'
PW = '비밀번호를 입력하세요'

print('로그인을 시작합니다.')
login(ID, PW)
print('로그인을 완료했습니다.')

time.sleep(2)

로그인을 시작합니다.
로그인을 완료했습니다.


# 리뷰 탐색

In [56]:
review_lst = []
review_num = 100        # 한 영화당 몇 개의 리뷰를 가져올 것인가?
scroll_num = 100        # 스크롤 다운을 몇 번 할 것인가?
rank = 0

for movie, movie_year in zip(m_list[:10], m_year[:10]):
    print(f'{rank+1}위. "{movie}" 리뷰 수집 시작!')

    # 입력 박스 선택
    if rank == 0:
        css_sel = '#root > div > div.css-1xm32e0 > header.css-6k8tqb > nav > div > div > ul > li.css-1c3wwgb > div > div > form > label > input'
    else:
        
        css_sel = '#root > div > div.css-1xm32e0 > header > nav > div > div > ul > li.css-1c3wwgb > div > div > form > label > input'
        
    input_box = driver.find_element(By.CSS_SELECTOR, css_sel)
    input_box.send_keys(movie)     # 검색어 입력
    input_box.send_keys(Keys.ENTER)   # 엔터키 입력

    time.sleep(3)
    
    # 해당 사이트로 이동
    titles = driver.find_elements(By.CLASS_NAME, 'css-31iyzt')        # 페이지에서 영화 제목 추출하기
    years = driver.find_elements(By.CLASS_NAME, 'css-1thqxgo')        # 페이지에서 영화 년도 추출하기

    for title, year in zip(titles, years):
        title_text = title.text
        title_year = year.text[0:4] 
        # print(title_text, movie, title_year, movie_year)
        
        if title_text == movie and title_year == str(movie_year):
            title.click()
            break
            
    time.sleep(2)

    # 감상 가능한 곳 
    try:
        sites = driver.find_elements(By.CSS_SELECTOR, '#root > div > div.css-1xm32e0 > section > div > div.css-10ofaaw > div > div > div > div:nth-child(1) > div.css-wpsvu8 > div.css-1nhig6u-RoundedCornerBlock-RoundedCornerBlock > div > section.w_exposed_cell.css-1impywp > div.css-usdi1z > div.css-9dnzub > div > div > div > ul > li')
        pos_sites = [site.text for site in sites]
        print(f'감상 가능한 곳: {pos_sites}')
    except:
        pass
        
    # 코멘트 더보기 누르기
    try:
        driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-1xm32e0 > section > div > div.css-10ofaaw > div > div > div > div:nth-child(1) > div.css-uvsgck > div > div > section:nth-child(5) > div:nth-child(1) > div > header > div > div > a').click()
        print('코멘트 페이지로 이동합니다', end=' ')
        time.sleep(2)
    except:
        driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-1xm32e0 > section > div > div.css-10ofaaw > div > div > div > div:nth-child(1) > div.css-uvsgck > div > div > section:nth-child(4) > div:nth-child(1) > div > header > div > div > a').click()
        print('코멘트 페이지로 이동합니다', end=' ')
        time.sleep(2)

    # 스크롤 내리기
    scr_down(scroll_num)            
    # doScrollDown(30)
    # driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.HOME)

    time.sleep(2)
    
    # 리뷰 긁어오기
    reviews = driver.find_elements(By.CSS_SELECTOR, '#root > div > div.css-1xm32e0 > section > section > div > div > div > ul > div')[:-1]

    print('Start!', end='')    
    for i, review in enumerate(reviews):
        
        # id = review.find_element(By.CLASS_NAME, 'css-1cvf9dk').text
        
        # score 읽기 
        try:
            score = review.find_element(By.CLASS_NAME, 'css-yqs4xl').text
        except:
            continue
        
        # text 읽기
        try:
            text = review.find_element(By.CLASS_NAME, 'css-1g78l7j').text       # 스포일러 아닐때 리뷰 읽기
        except:
            text = review.find_element(By.CLASS_NAME, 'css-1rdcd6n').text       # 스포일러일때 리뷰 읽기
            print('!', end='')
        
        # print(review_cnt, id, score, text)
        
        # 데이터셋 만들기
        review_lst.append([movie, text, score])
        
        # 메세지 출력
        if (i + 1) % 10 == 0:
            if (i + 1) != 10 and (i + 1) % 200 == 10:
                print(f'\n> {i + 1}', end=' ')
            else:
                print(f'> {i + 1}', end=' ')
        
        if (i + 1) == review_num:
            print('> END!')
            break

    rank += 1
    print(f'리뷰 {i + 1}개 수집 완료!')
    print('-'*80)

1위. "명량" 리뷰 수집 시작!
감상 가능한 곳: ['왓챠', '티빙']
코멘트 페이지로 이동합니다 >>>>> Scroll Down End!
Start!!> 10 > 20 > 30 > 40 > 50 > 60 > 70 > 80 > 90 !> 100 > END!
리뷰 100개 수집 완료!
--------------------------------------------------------------------------------
2위. "극한직업" 리뷰 수집 시작!
감상 가능한 곳: ['왓챠', '티빙']
코멘트 페이지로 이동합니다 >>>>> Scroll Down End!
Start!> 10 !> 20 > 30 > 40 > 50 !> 60 !> 70 > 80 !> 90 > 100 > END!
리뷰 100개 수집 완료!
--------------------------------------------------------------------------------
3위. "신과함께-죄와 벌" 리뷰 수집 시작!
감상 가능한 곳: ['왓챠', '넷플릭스', '웨이브', '티빙']
코멘트 페이지로 이동합니다 >>>>> Scroll Down End!
Start!!> 10 > 20 > 30 > 40 > 50 > 60 > 70 !!> 80 > 90 > 100 > END!
리뷰 100개 수집 완료!
--------------------------------------------------------------------------------
4위. "국제시장" 리뷰 수집 시작!
감상 가능한 곳: ['왓챠', '티빙']
코멘트 페이지로 이동합니다 >>>>> Scroll Down End!
Start!> 10 > 20 > 30 > 40 !> 50 > 60 > 70 > 80 > 90 > 100 > END!
리뷰 100개 수집 완료!
--------------------------------------------------------------------------------
5위. 

In [58]:
review_data = pd.DataFrame(review_lst, columns=['movie', 'review', 'score'])
#review_data.to_csv('./review_data_watcha.csv')
review_data

,movie,review,score
0,명량,"시종일관 강약없는 비장은 실소를 유발\n 박영태님, 신윤수님과 함께.",2.0
1,명량,최민식이 이순신으로 나와서 일본사람 잡는데 대한민국사람이라면 감동해야지?? 라고 강...,1.0
2,명량,어쨌든 해전 부분은 인상적.,2.5
3,명량,"칼의 노래도 아니고,\n300도 아니고.\n뻔한 연출에 과한 연극.\n성웅을 그린다...",1.5
4,명량,,2.0
...,...,...,...
981,7번방의 선물,울리는 데 성공했다고 좋아하지 마라\n울었다고 너를 인정하는 건 아니니까,1.0
982,7번방의 선물,"류승룡과 갈소원의 연기로 관객 울리기, 몰이에는 아주 성공적이었으나, 질적으로는 퇴...",2.5
983,7번방의 선물,이런 천박한 감성팔이 그만하세요;;\n억지스러워서 겨우 봤네,0.5
984,7번방의 선물,내 머리 커서,2.0
